<a href="https://colab.research.google.com/github/ljdyer/text-data-cleaner/blob/main/clean_ted_talks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning TED Talks text data

**Goal**: To remove all characters other than a-z, A-Z, 0-9, space, comma, and period, whilst staying as close as possible to the way a human would format the text using these characters in order to generate high quality trianing data for our ML project. Due to the size of the dataset, it is not possible to confirm every change one-by-one, and it is inevitable that some weirdness will be introduced, but I attempt to minimize this as much as realistically possible.

## 1. Initial set up

Import the helper functions in the 'text data cleaner' library that I developed.

Documentation is at https://github.com/ljdyer/text-data-cleaner, and each of the functions is demonstrated below.

In [1]:
url = 'https://raw.githubusercontent.com/ljdyer/text-data-cleaner/main/text_data_cleaner.py'
!wget --no-cache -backups=1 'text_data_cleaner.py' {url}

Continuing in background, pid 1073.


In [2]:
import text_data_cleaner
from text_data_cleaner import *

The following is to set Google Colab to wrap long lines in cell output for this notebook.

In [3]:
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## 2. Load text data into pandas dataframe

You will need to change the following to the path for your working folder and dataset file.

In [4]:
SOURCE_CSV = 'drive/MyDrive/Group Assignment/TED_Talks/ted_talks_en.csv'
SAVE_FOLDER = 'drive/MyDrive/Group Assignment/TED_Talks'

Load the file into a pandas dataframe

In [5]:
import pandas as pd

# Hide chained assignment warning
pd.options.mode.chained_assignment = None   # default='warn'

# Mount drive if loading from Google Drive (not necessary if working on local
# hard disk)
from google.colab import drive
drive.mount('/content/drive')

# Read from csv file in pandas dataframe.
ted_talks = pd.read_csv(SOURCE_CSV)

# Copy the column with the text into a new dataframe (you can keep more columns
# if you wish, but they were not necessary for me)
transcripts = ted_talks[['transcript']].copy()

# Name column 'Text'
# This is the default value for the helper functions. If you want to call it
# something else, you will need to pass that name as the 'text_column_name'
# parameter to each helper function.
transcripts.columns = ['Text']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check that the dataframe contains the information you expect.

In [6]:
transcripts.head()

,Text
0,"Thank you so much, Chris. And it's truly a gre..."
1,"About 10 years ago, I took on the task to teac..."
2,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,If you're here today — and I'm very happy that...
4,Good morning. How are you? (Audience) Good. It...


# 3. Clean the data

We can use the `show_doc__and_word_counts` function to check how much text is in the dataframe. We should compare the size after each replacement step with the initial size, as a dramatic drop in size would indicate that something has gone wrong.

In [7]:
show_doc_and_word_counts(transcripts)

7196367 words in 4005 documents (rows).


We can use the `show_prohibited_chars` function to display information about 'prohibited' characters that are present in the data. The prohibited characters can be specified, but the default is set to `r'[^A-Za-z0-9 \.,]'` which is appropriate for this project. Usually only the top ten prohibited characters are displayed, but here I specify `print_all=True` to see all of the prohibited characters.

In [8]:
show_prohibited_chars(transcripts, print_all=True)

Total of 457235 occurrences of 148 prohibited characters in dataframe.
, ć, ğ, о, ), £, τ, Ō, ي, π, ], ة, à, ẹ, Ż, :, …, Ť, (, ¡, ه, ل, ì, =, В, ē, *, ？, Ă, ⁰, —, è, ¿, ë, Č, $, /, อ, “, š, ́, +, ě, ك, ø, ç, ', È, ū, ², [, º, ?, ê, 文, ú, â, , ", ô, ọ, د, ♪, ō, ы, , म, 说, !, ’, ö, í, ̀, ร, á, ”, \, é, प, 贸, 会, س, Á, 吗, Ģ, े, ð, ü, –, 你, 中, ä, ⅔, ḥ, ;, ‎, Ö, À, 送, ∇, @, -, ˚, ò, 好, î, &, €, ï, œ, ©, û, #, ع, ص, ¢, ʾ, ā, ǔ, ī, ^, 葱, Ç, र, ط, •, Å, ù, ¹, Ü, æ, ्, ñ, _, ó, م, É, ǐ, ย, ã, ن, و, ı, ♫, ‘, ่, ǒ, %
Most common (up to 10 displayed):  ' (190032), " (57163), - (37870), ? (36933), — (33846), ) (25803), ( (25781), : (23962), ; (5858), ’ (4105)


OK, we've got lots of work to do! This is where the fun part starts. We can use `preview_regex_replace` to check the effect that applying a regex find and replace operation would have on the data, before we actually go ahead and apply it. Bear in mind that this is only a preview, and the dataframe will not be changed until we call `regex_replace` later.

Imagine for example that I was considering removing all brackets from the data. I would run the following:

In [9]:
preview_regex_replace(r'[\(|\)]', '', transcripts)

Row index,Match number,Before,After
1853,10/20,...amned clitoris. (Laughter) What is a woman without ...,...amned clitoris. Laughter What is a woman without ...
1058,13/44,...d the story go together. (Laugther) If you think ab...,...d the story go together. Laugther If you think ab...
1665,4/8,"...and not a park. (Applause) And you know what, it mi...","...and not a park. Applause And you know what, it mi..."
1973,5/18,... governments. (Laughter) (Applause) I don't have to...,... governments. Laughter Applause I don't have to...
1489,28/42,...e different keys. (Tuning) Yeah. I'm also going to ...,...e different keys. Tuning Yeah. I'm also going to ...
3131,14/20,"...hard to bear (Synth music) I'm so dizzy, the air is...","...hard to bear Synth music I'm so dizzy, the air is..."
1819,2/2,... so. Thank you. (Applause),... so. Thank you. Applause
226,28/40,"...hter) Get some. (Applause) So, this is really where...","...hter Get some. Applause So, this is really where..."
324,3/20,(Music) (Applause) Thank you for b...,Music Applause Thank you for b...
1914,18/20,"...o it's perfect. (Laughter) So now, what would happe...","...o it's perfect. Laughter So now, what would happe..."


Total of 51584 matches in 3403 documents (rows).


10 random examples from a total of 51,584 matches were shown. We can tell from this that the vast majority of the brackets are used around phrases like 'Laughter' and 'Applause' that do not actually form part of the sentence. Removing only the brackets leads to weird sentences like "Laughter the four of us met...", as we can see on the right-hand side of the table above. It is better to remove the contents of the brackets as well to keep the sentences coherent and enable our models to learn correctly based on grammatical structure from POS tags, etc.

I'm not quite confident enough to apply that general rule for all the data yet though, so I'll start by removing '(Applause)' and '(Laughter)' only to see what's left. I'll preview those first, adjusting my find and replace regexes as necessary if I don't get the intended result the first time.

In [10]:
preview_regex_replace(r'\(Laughter\)', '', transcripts)
preview_regex_replace(r'\(Applause\)', '', transcripts)

Row index,Match number,Before,After
660,32/32,.... It just wouldn't work. (Laughter) But it's not only about ...,.... It just wouldn't work. But it's not only about ...
344,7/8,"...try speed it up a little (Laughter) Moderator: Eva, we have ...","...try speed it up a little Moderator: Eva, we have ..."
79,2/2,...s or closed auditoriums. (Laughter) We filter through some s...,...s or closed auditoriums. We filter through some s...
1941,6/19,"...t Get All Pole-emical"" — (Laughter) — and it's in Beekman Ha...","...t Get All Pole-emical"" — — and it's in Beekman Ha..."
535,1/2,... anything that he wrote. (Laughter) This is how he wrote mos...,... anything that he wrote. This is how he wrote mos...
2607,3/4,"...useguest finally leaves. (Laughter) Now, you or I might feel...","...useguest finally leaves. Now, you or I might feel..."
433,5/6,...hat Enron Stadium thing. (Laughter) Fine arts and education ...,...hat Enron Stadium thing. Fine arts and education ...
169,8/10,"...ng, but I also practice. (Laughter) 4 am in the morning. (La...","...ng, but I also practice. 4 am in the morning. (La..."
1247,2/3,...well — that's the story. (Laughter) The next painting I'm go...,...well — that's the story. The next painting I'm go...
892,7/9,"...2,000 bars of chocolate. (Laughter) Wait — The same study fo...","...2,000 bars of chocolate. Wait — The same study fo..."


Total of 13949 matches in 2500 documents (rows).


Row index,Match number,Before,After
2023,2/3,"...h Wachter, come onstage. (Applause) First, tell me just a de...","...h Wachter, come onstage. First, tell me just a de..."
2741,1/2,"...ation I made. (Laughter) (Applause) I assure you, people hav...","...ation I made. (Laughter) I assure you, people hav..."
2062,8/9,"...MAN, NO CRY."" (Laughter) (Applause) She said, ""MY BANKER WIL...","...MAN, NO CRY."" (Laughter) She said, ""MY BANKER WIL..."
2757,3/5,... bipartisan — (Laughter) (Applause) And invited me to give c...,... bipartisan — (Laughter) And invited me to give c...
2780,3/9,...Drone buzzes) (Laughter) (Applause) To a machine that helps ...,...Drone buzzes) (Laughter) To a machine that helps ...
648,7/7,...l incentives. Thank you. (Applause),...l incentives. Thank you.
1228,1/1,...nk you so much. Shukran! (Applause),...nk you so much. Shukran!
263,1/2,"...car, is origami. (Video) (Applause) Just to show you, this r...","...car, is origami. (Video) Just to show you, this r..."
973,2/3,...n this slide. (Laughter) (Applause) JM: Now there are many o...,...n this slide. (Laughter) JM: Now there are many o...
3498,1/1,...ry and world. Thank you. (Applause),...ry and world. Thank you.


Total of 7534 matches in 3242 documents (rows).


I am satisfied with the results here, so I'll go ahead and use the `regex_replace` function. This takes a list of tuples of (find, replace) regex pairs as its first parameter.

In [11]:
transcripts = regex_replace([(r'\(Laughter\)', ''), (r'\(Applause\)', '')], transcripts)

Done.
7174922 words in 4005 documents (rows).


Let's see what bracketed expressions remain.

In [12]:
preview_regex_replace(r'[\(|\)]', '', transcripts)

Row index,Match number,Before,After
3486,204/462,"...biscuit, biscuit, biscuit) has been to use the crea...","...biscuit, biscuit, biscuit has been to use the crea..."
3486,430/462,... unlock new ways (biscuit) of seeing and understand...,... unlock new ways biscuit of seeing and understand...
1064,6/12,"..., dying fall ... (Singing) Da da dee, da da da da, ...","..., dying fall ... Singing Da da dee, da da da da, ..."
777,1/4,...ight in the face: 58 Gs. (Music) In case you missed...,...ight in the face: 58 Gs. Music In case you missed...
3123,20/32,...ves. (Applause and cheers) Or — Or Lauren Underwood...,...ves. Applause and cheers Or — Or Lauren Underwood...
3046,9/24,"...ello, you think of this. (Plays Bach Cello Suite No...","...ello, you think of this. Plays Bach Cello Suite No..."
3486,264/462,"...hours. (hedgehog, biscuit) They can happen anywhere...","...hours. hedgehog, biscuit They can happen anywhere..."
1527,1/8,...That's a different show. (Squeaking) That's rather ...,...That's a different show. Squeaking That's rather ...
2563,4/4,...ome to the tribe. (Cheers),...ome to the tribe. Cheers
1115,8/16,...what you get. (Voice Over) Woody: What do you think...,...what you get. Voice Over Woody: What do you think...


Total of 8618 matches in 1057 documents (rows).


We might decide to just remove them all at this stage. I went a little more granular and found some cases where it was better to remove the outer brackets but keep the text inside.

From here on it is just a process of repeated trial and error and making a judgement call at each stage as to whether to go ahead with the replacement. It is not possible to avoid some weirdness, so if around 9 out of the 10 results displayed leaves more or less what we would hope for in our data, it's usually OK to go ahead and make the change.

To keep the notebook from becoming very long, I stored my 'confirmed regex pairs' in a variable and called `replace_regex` from the same cell each time before previewing the next replacement from the cell below. I ended up with the following.

You may be able to reuse some of these, but bear in mind that the appropriate replacements will be different for each dataset so try to preview each replacement before applying it if possible.

In [13]:
confirmed_regex_pairs = [
    ('[’|‘|ʾ]', "'"),
    ('[“|”]', '"'),
    ('…', "..."),
    (r"\?", r"."),
    (r"!", r"."),
    (r'\((\w)\)', r"\1"),                 # Brackets around a single character
    (r'(?<![,:]) \([^\)]+\)(,)', r'\1'),  # Brackets before a comma
    (r'\([^\)]+\)', r''),                 # Other bracketed expressions
    (r'\[([a-z]\S+)\]', r'\1'),           # Lowercase word in square brackets
    (r'\[([a-z]\S+ [a-z]\S+)\]', r'\1'),  # Two lowercase words in square brackets
    (r'\[[^\]]*\]', ''),                  # Everything else in square brackets
    (r'[A-Z]+:', r''),                    # Speaker initials
    (r'(([A-Z][a-z]+ )+([A-Z][a-z]+)+:)', ''),   # Speaker names
    (r'Video:|Narrator:|All:|Man:|Woman:', ''),  # Speaker indicators
    (r'Voice [0-9]:', ''),         
    (r'Woman [0-9]:', ''),
    (r'Man [0-9]:', ''),
    (r'D(octor )?[0-9]:', ''),
    (r'SC [0-9]*:', ''),
    (r'([0-9]+):([0-9]+)', r'\1 \2'),     # Times
    (r'([a-z]): ([a-z])', r'\1, \2'),     # Colon between two lowercase characters
    (r'([a-z]): ([A-Z])', r'\1. \2'),     # Colon between uppercase and lowercase characters
    (r':', r','),                         # Remaining colons
    (r"'", r''),                          # Apostrophes
    (r'"', r''),                          # Quotation marks
    (r' *— *', r', '),                    # Em dashes
    (r'50-50', r'50 50'),
    (r'24-7', r'24 7'),
    (r' ([0-9]+)-([0-9]+) ', r' \1 to \2 '),  # Number ranges
    (r' (re|un|pre)-(\w)', r' \1\2'),     # Prefix with hyphen
    (r'([^,]) - ', r'\1, '),              # Hyphen with spaces around
    (r', - ', r', '),                     # Hyphen after comma + space
    (r'-', r' '),                         # Remaining hyphens
    (r'%', r' percent'),                  # Percent
    (r'\+', r' plus '),                   # Plus
    (r'([0-9]) *\^ *([0-9])', r'\1 to the power of \2'),   # 'To the power of'
    (r'([0-9]) *x *([0-9])', r'\1 times \2'),    # Times
    (r'=', r' equals '),                  # Equals
    (r' @(\w)', r' \1'),                  # @ in Twitter handles
    (r'(\w)@(\w)', r'\1 at \2'),          # @ in email addresses
    (r'(\w)\.com', r'\1 dot com'),        # dot com
    (r' #(\w)', r' \1'),                  # hashtags/numbers
    (r'; ([A-Z])', r'. \1'),              # Semicolon before capital letters
    (r';', r','),                          # All other semicolons
    (r'\$', r'USD '),                     # Dollars
    (r'&', r' and '),                     # Ampersands
    (r'/', r' '),
    (r'___+', r'blank'),                   # ___ to indicate 'blank'
]

transcripts = regex_replace(confirmed_regex_pairs, transcripts)

Done.
7153757 words in 3997 documents (rows).


I can confirm from the above that the decrease in the size of the data is within the limits of what I would expect.

There are still quite a lot of prohibited characters left in the data.

In [14]:
show_prohibited_chars(transcripts)

Total of 6417 occurrences of 124 prohibited characters in dataframe.
Most common (up to 10 displayed):  ♫ (3880), ♪ (690), é (504), – (377), í (119), á (114), ö (59), ó (53), ã (40), ç (38)


The `normalize_unicode` function replaces accented characters with their ASCII equivalents (e.g. 'e' for 'é') and removes any non-ASCII characters that do not have ASCII equivalents (like '送').

In [15]:
transcripts = normalize_unicode(transcripts)
show_prohibited_chars(transcripts)

Total of 39 occurrences of 9 prohibited characters in dataframe.
Most common (up to 10 displayed):  ) (15), # (7), * (4), _ (4), ^ (3), \ (2), @ (2), ( (1), ? (1)


We've removed a lot of prohibited characters! That's good enough so we'll just replace the remaining ones with spaces.

In [16]:
transcripts = regex_replace([(r'[^A-Za-z0-9 \.,]', '')], transcripts)
show_prohibited_chars(transcripts)

Done.
7149001 words in 3997 documents (rows).
Total of 0 occurrences of 0 prohibited characters in dataframe.
Most common (up to 10 displayed):  


## 3. Save the cleaned data

This is a good place to take a look at the data and check it is how we expect.

In [17]:
transcripts.head()

,Text
0,"Thank you so much, Chris. And its truly a grea..."
1,"About 10 years ago, I took on the task to teac..."
2,"Hello voice mail, my old friend. Ive called f..."
3,"If youre here today, and Im very happy that yo..."
4,Good morning. How are you. Good. Its been grea...


Looking good! I'll now create the columns for the different types of data we may need need to test different stages of our models. For the overall pipeline, `all_cleaned` will be the input (X) and `no_spaces` will be the output (y).

You can use the following code as-is once you have finished cleaning your data.

In [18]:
# Rename 'Text' column to 'all_cleaned'
transcripts.columns = ['all_cleaned']
# Remove commas and periods and store in a new column called 'no_punctuation'
transcripts['no_punctuation'] = regex_replace([(r'[\,\.]', '')], transcripts.copy(), text_column_name='all_cleaned')['all_cleaned']
# Lowercase everything and store in a new column called 'lower'
transcripts['lower'] = transcripts['no_punctuation'].apply(lambda x: x.lower())
# Remove spaces and store in a new column called 'no_spaces'
transcripts['no_spaces'] = regex_replace([(r' ', '')], transcripts.copy(), text_column_name='lower')['lower']

transcripts.head()

Done.
7146415 words in 3997 documents (rows).
Done.
3997 words in 3997 documents (rows).


,all_cleaned,no_punctuation,lower,no_spaces
0,"Thank you so much, Chris. And its truly a grea...",Thank you so much Chris And its truly a great ...,thank you so much chris and its truly a great ...,thankyousomuchchrisanditstrulyagreathonortohav...
1,"About 10 years ago, I took on the task to teac...",About 10 years ago I took on the task to teach...,about 10 years ago i took on the task to teach...,about10yearsagoitookonthetasktoteachglobaldeve...
2,"Hello voice mail, my old friend. Ive called f...",Hello voice mail my old friend Ive called for...,hello voice mail my old friend ive called for...,hellovoicemailmyoldfriendivecalledfortechsuppo...
3,"If youre here today, and Im very happy that yo...",If youre here today and Im very happy that you...,if youre here today and im very happy that you...,ifyoureheretodayandimveryhappythatyouareyouvea...
4,Good morning. How are you. Good. Its been grea...,Good morning How are you Good Its been great h...,good morning how are you good its been great h...,goodmorninghowareyougooditsbeengreathasntitive...


Save it, and we're done!

In [19]:
import os
transcripts.to_csv(os.path.join(SAVE_FOLDER, 'TED_TRAIN_TEST.csv'))